In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/sentiment_analysis
!ls

Mounted at /content/drive
/content/drive/MyDrive/sentiment_analysis
BoW_Sentiment_Model.pkl		     RestaurantReviews_HistoricDump.tsv
Classifier_Sentiment_Model	     Sentiment_Analysis_Model.ipynb
Predicted_Sentiments_Fresh_Dump.tsv  Sentiment_Predictor.ipynb
RestaurantReviews_FreshDump.tsv


In [3]:
dataset = pd.read_csv('RestaurantReviews_HistoricDump.tsv', delimiter = '\t', quoting = 3)

In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
corpus=[]
for i in range(0, 900):
  review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=2500, ngram_range=(1,2))
X = tfidf.fit_transform(corpus)
y = dataset.iloc[:, -1].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.01, random_state = 42)


from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

import pickle, joblib
pickle.dump(tfidf, open('TFIDF_Sentiment_Model.pkl', 'wb'))
joblib.dump(classifier, 'Classifier_Sentiment_Model')


y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)


0.8888888888888888